# Phenocycler Data Preprocessing and Quality Control

This notebook handles the initial preprocessing steps for Phenocycler spatial proteomics data:
- Loading data from qptiff files or CSV (QuPath export)
- Quality control metrics
- Filtering low-quality cells and markers
- Normalization
- Dimensionality reduction (PCA, UMAP)

**Input:** Raw qptiff files or CSV files from QuPath

**Output:** Preprocessed AnnData object ready for downstream analysis

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from utils.analysis_utils import (
    load_qupath_csv,
    calculate_qc_metrics,
    filter_cells_and_markers,
    normalize_and_hvg,
    run_standard_workflow,
    create_summary_report
)

# Set plotting parameters
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)
sns.set_style('whitegrid')

print(f"Scanpy version: {sc.__version__}")
print(f"Squidpy version: {sq.__version__}")

## 1. Define Paths and Parameters

In [ ]:
# Define paths
DATA_DIR = Path("../data/raw")  # Directory containing CSV files
OUTPUT_DIR = Path("../data/processed")
FIGURES_DIR = Path("../figures/01_preprocessing")

# Create output directories if they don't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Sample name
SAMPLE_NAME = "phenocycler_sample_01"  # Update with your sample name

# Data format: 'csv' or 'qptiff'
DATA_FORMAT = 'csv'  # Change to 'qptiff' if using qptiff files

# QC thresholds
MIN_MARKERS = 5  # Minimum markers per cell
MIN_CELLS = 3   # Minimum cells per marker
MAX_BACKGROUND = 0.2  # Maximum background threshold
MIN_INTENSITY = 10  # Minimum total intensity per cell

# Normalization method
NORMALIZATION = 'zscore'  # Options: 'zscore', 'log', 'arcsinh'

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Figures directory: {FIGURES_DIR}")

## 2. Load Phenocycler Data

Load the Phenocycler data from CSV export (QuPath) or qptiff file. 

For CSV format, you should have:
- `expression.csv`: Cell x protein expression matrix
- `coordinates.csv`: Cell spatial coordinates (x, y)
- `metadata.csv` (optional): Additional cell metadata

In [ ]:
# Load data based on format
if DATA_FORMAT == 'csv':
    # Load from QuPath CSV export
    adata = load_qupath_csv(
        csv_dir=DATA_DIR,
        sample_name=SAMPLE_NAME,
        expression_file=f"{SAMPLE_NAME}_expression.csv",
        coordinates_file=f"{SAMPLE_NAME}_coordinates.csv",
        metadata_file=f"{SAMPLE_NAME}_metadata.csv"
    )
elif DATA_FORMAT == 'qptiff':
    # Load from qptiff file
    from utils.analysis_utils import load_qptiff_data
    qptiff_file = DATA_DIR / f"{SAMPLE_NAME}.qptiff"
    adata = load_qptiff_data(qptiff_file, SAMPLE_NAME)
else:
    raise ValueError(f"Unknown data format: {DATA_FORMAT}")

# Display basic information
print(f"\nDataset shape: {adata.shape}")
print(f"Number of cells: {adata.n_obs}")
print(f"Number of markers: {adata.n_vars}")
print(f"\nMarkers: {list(adata.var_names[:10])}...")

# Check if spatial coordinates are present
if 'spatial' in adata.obsm:
    print(f"\n✓ Spatial coordinates loaded: {adata.obsm['spatial'].shape}")
else:
    print("\n⚠ Warning: No spatial coordinates found")

## 3. Quality Control Metrics

In [ ]:
# Calculate QC metrics
adata = calculate_qc_metrics(adata, background_threshold=MAX_BACKGROUND)

# Display QC statistics
print("\nCell-level QC metrics:")
print(adata.obs[['n_markers', 'total_intensity', 'mean_intensity']].describe())

print("\nMarker-level QC metrics:")
print(adata.var[['n_cells', 'mean_expression']].describe())

## 4. QC Visualizations

In [ ]:
# Create QC plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Markers per cell distribution
axes[0, 0].hist(adata.obs['n_markers'], bins=50, edgecolor='black')
axes[0, 0].axvline(MIN_MARKERS, color='red', linestyle='--', label=f'Min threshold={MIN_MARKERS}')
axes[0, 0].set_xlabel('Number of markers per cell')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].set_title('Markers per Cell')

# Total intensity distribution
axes[0, 1].hist(adata.obs['total_intensity'], bins=50, edgecolor='black')
axes[0, 1].axvline(MIN_INTENSITY, color='red', linestyle='--', label=f'Min threshold={MIN_INTENSITY}')
axes[0, 1].set_xlabel('Total intensity')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].set_title('Total Intensity per Cell')

# Mean intensity distribution
axes[0, 2].hist(adata.obs['mean_intensity'], bins=50, edgecolor='black')
axes[0, 2].set_xlabel('Mean intensity')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Mean Intensity per Cell')

# Cells per marker distribution
axes[1, 0].hist(adata.var['n_cells'], bins=50, edgecolor='black')
axes[1, 0].axvline(MIN_CELLS, color='red', linestyle='--', label=f'Min threshold={MIN_CELLS}')
axes[1, 0].set_xlabel('Number of cells per marker')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()
axes[1, 0].set_title('Cells per Marker')

# Mean expression per marker
axes[1, 1].hist(adata.var['mean_expression'], bins=50, edgecolor='black')
axes[1, 1].set_xlabel('Mean expression')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Mean Expression per Marker')

# Marker expression heatmap (top expressed)
top_markers = adata.var.nlargest(20, 'mean_expression').index
marker_means = [adata[:, marker].X.mean() for marker in top_markers]
axes[1, 2].barh(range(len(top_markers)), marker_means)
axes[1, 2].set_yticks(range(len(top_markers)))
axes[1, 2].set_yticklabels(top_markers, fontsize=8)
axes[1, 2].set_xlabel('Mean expression')
axes[1, 2].set_title('Top 20 Markers by Expression')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'qc_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("QC plots saved!")

## 5. Spatial QC (if coordinates available)

In [ ]:
if 'spatial' in adata.obsm:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot spatial distribution of cells
    coords = adata.obsm['spatial']
    
    # Number of markers
    sc1 = axes[0].scatter(coords[:, 0], coords[:, 1], 
                          c=adata.obs['n_markers'], 
                          cmap='viridis', s=1, alpha=0.5)
    axes[0].set_title('Markers per Cell (Spatial)')
    axes[0].set_xlabel('X coordinate')
    axes[0].set_ylabel('Y coordinate')
    plt.colorbar(sc1, ax=axes[0])
    
    # Total intensity
    sc2 = axes[1].scatter(coords[:, 0], coords[:, 1], 
                          c=adata.obs['total_intensity'], 
                          cmap='viridis', s=1, alpha=0.5)
    axes[1].set_title('Total Intensity (Spatial)')
    axes[1].set_xlabel('X coordinate')
    axes[1].set_ylabel('Y coordinate')
    plt.colorbar(sc2, ax=axes[1])
    
    # Cell density
    axes[2].hexbin(coords[:, 0], coords[:, 1], gridsize=50, cmap='viridis')
    axes[2].set_title('Cell Density')
    axes[2].set_xlabel('X coordinate')
    axes[2].set_ylabel('Y coordinate')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'spatial_qc.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Skipping spatial QC (no coordinates available)")

## 6. Filter Low-Quality Cells and Markers

In [ ]:
# Filter cells and markers
adata_filtered = filter_cells_and_markers(
    adata,
    min_markers=MIN_MARKERS,
    min_cells=MIN_CELLS,
    min_intensity=MIN_INTENSITY,
    max_background_ratio=0.5
)

print(f"\nFiltering summary:")
print(f"  Cells: {adata.n_obs} → {adata_filtered.n_obs} ({100*adata_filtered.n_obs/adata.n_obs:.1f}%)")
print(f"  Markers: {adata.n_vars} → {adata_filtered.n_vars} ({100*adata_filtered.n_vars/adata.n_vars:.1f}%)")

## 7. Normalization and Feature Selection

In [ ]:
# Normalize data
adata_norm = normalize_and_hvg(
    adata_filtered,
    method=NORMALIZATION,
    n_top_markers=min(30, adata_filtered.n_vars),  # Adjust based on total markers
    target_sum=10000
)

print(f"\nNormalization complete using {NORMALIZATION} method")
if 'highly_variable' in adata_norm.var.columns:
    print(f"Highly variable markers: {adata_norm.var['highly_variable'].sum()}")

## 8. Dimensionality Reduction and Clustering

In [ ]:
# Run standard workflow (PCA, UMAP, clustering)
n_pcs = min(30, adata_norm.n_vars - 1)  # Adjust based on number of markers

adata_processed = run_standard_workflow(
    adata_norm,
    n_pcs=n_pcs,
    n_neighbors=15,
    resolution=1.0
)

print("\nDimensionality reduction and clustering complete!")

## 9. Visualization

In [ ]:
# PCA variance explained
fig, ax = plt.subplots(figsize=(8, 5))
sc.pl.pca_variance_ratio(adata_processed, log=True, n_pcs=n_pcs, ax=ax, show=False)
plt.savefig(FIGURES_DIR / 'pca_variance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# UMAP colored by clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sc.pl.umap(adata_processed, color='leiden', ax=axes[0], show=False, legend_loc='on data')
axes[0].set_title('UMAP - Leiden Clusters')

sc.pl.umap(adata_processed, color='n_markers', ax=axes[1], show=False)
axes[1].set_title('UMAP - Number of Markers')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'umap_overview.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot top markers on UMAP
if 'highly_variable' in adata_processed.var.columns:
    top_markers = adata_processed.var[adata_processed.var['highly_variable']].nlargest(6, 'mean_expression').index
    sc.pl.umap(adata_processed, color=top_markers, ncols=3, cmap='viridis', save='_top_markers.png')
    print(f"Top markers plotted: {list(top_markers)}")

## 10. Save Processed Data

In [ ]:
# Save preprocessed data
output_file = OUTPUT_DIR / f"{SAMPLE_NAME}_preprocessed.h5ad"
adata_processed.write_h5ad(output_file)
print(f"\nPreprocessed data saved to: {output_file}")

# Create summary report
summary_file = OUTPUT_DIR / f"{SAMPLE_NAME}_preprocessing_summary.csv"
summary = create_summary_report(adata_processed, SAMPLE_NAME, summary_file)
print(f"\nSummary report saved to: {summary_file}")
print("\nSummary:")
print(summary)

## 11. Next Steps

The preprocessed data is now ready for:
1. **Cell type annotation** (02_phenotyping.ipynb)
2. **Spatial analysis** (03_spatial_analysis.ipynb)
3. **Comparative analysis** (04_group_comparisons.ipynb)

Key outputs:
- Preprocessed AnnData: `{SAMPLE_NAME}_preprocessed.h5ad`
- QC figures in: `figures/01_preprocessing/`
- Summary statistics: `{SAMPLE_NAME}_preprocessing_summary.csv`